Velkých jazykových modelů (large language models, LLM) dnes existuje  velké množství. Variabilita je možná už tak velká, že se v nabídce člověk snadno ztratí. Aby se to nestalo i mně, začal jsem si dělat poznámky, které potenciální zájemce nalezne v tomto souboru. Přirozeně ale vyvstává otázka, zda zastarají za 6 nebo 12 měsíců...  
První dělení LLM jde po ose proprietární modely, které lze použít jen skrze API jejich mateřské firmy (GPT, Claud, Gemini atd.), versus modely více či méně otevřené. Věnovat se zde budeme pouze druhé skupině.

## Dělení dle quantization

### "Čisté" modely
Jedná se o model v té podobě, v jaké byl natrénován. Papírově má lepší (současně přesvědčivější a pravdivější) výkon než dále zmíněné možnosti, což je ale vykoupeno většími hardwarovými nároky. Dřív měly tyto modely podobu adresáře s hromadou malých souborů a jedním velkým pytorchím picklem. Dnes byl ve většině případů pickle nahrazen safetensorem. Safetensory jsou údajně o [něco rychlejší než pickly](https://huggingface.co/docs/safetensors/speed) a použitelné i v jiných jazycích než Python. Nejdůležitější roli pro nás ale hraje skutečnost, že jsou bezpečnější - oproti picklům si lze jen obtížně představit, že by do nich někdo vložil škodlivý kód, který by se při jejich načtení spustil. Více informací například [zde](https://github.com/huggingface/safetensors/discussions/111).

### GGUF
Zkratka znamená GPT Generated Unified Format. Ty první dvě Gčka možná též představují iniciály autora - Georgi Greganov. Jedná se o stejného člověka, který je autorem llamma.cpp (viz dále). Výhoda formátu tkví ve faktu, že dokáže pracovat jak s GPU (s tím, že se model načte do VRAM), tak s CPU (model se načte do RAM). Možná je i kombinace obojího, rychlost v takovém případě ale dle internetu závratná nebude (i v porovnání s čístým CPU přístupem).  
Běh na CPU je nepřekvapivě pomalejší než na GPU. Nicméně zde se dostáváme k další výhodě GGUF modelů - dovolují použít menší datový typ pro uložení vah, než byl v modelu původním. V takovém případě mluvíme o "quantization". De facto se tak provede něco na způsob zaokrouhlení. Čím je toto zaokrouhlení agresivnější, tím má GGUF model menší velikost a pracuje rychleji, bohužel je ale také hloupější.
Oproti čistým modelům tvoří GGUF model jeden jediný soubor. Z jeho jména pak můžeme odvodit, jak silná quantizace byla. Se seznamem sufixů i s technický vysvětlením se můžeme seznámit [na spodku této stránky](https://huggingface.co/docs/hub/gguf), trochu srozumitelnější vysvětlení se nalézá [tady](https://www.reddit.com/r/LocalLLaMA/comments/1ba55rj/overview_of_gguf_quantization_methods/). Obecně Q8 je z hlediska výkonnosti téměř na úrovni původního modelu, Q2 bývá obvykle hodně dada, Q4/Q5 zase zlatý střed (objevily se i vědecké články o Q1, ale na vlastní PC jsem ještě s něčím podobným do styku nepřišel). Písmeno "K" představuje novější typ quantizace, S, M a L (small, medium, large) zase podrobnější úroveň quantizace na (z hlediska primární úrovně quantizace) - pro S model byla quantizace silnější než pro L model.  
GGUF formát by měl být rozhodně bezpečnější než pickle. Nicméně co se týče implementace v llama.cpp, svého času se objevila zpráva o [security díře](https://www.reddit.com/r/LocalLLaMA/comments/1bist4o/alert_gguf_security_advisory/). Ta je už naštěstí dávno patchnutá a co se ví, nikdo tuto díry nevyužil.  
V dnešní době se obvykle GGUF mutace modelů objeví současně s modelem původním, obvykle na stejném hugging face repozitáři.

### GGML
Zkratka znamená GPT-Generated Model Language (anebo možná Georgi Greganov Model Language). Jedná se o předchůzce GGUF, tj. dnes se už nepoužívá.

### GPTQ, AWQ, EXL2
Jedná se o quantizované modely využívající pouze GPU. Nejstarší bylo GPTQ, nejnovější EXL2. Obecně čím novější formát, tím větší rychlost - prý. Jelikož používám primárně GGUF, tak o výhodách a problémech těchto GPU-only mutací modelů příliš nevím. Zdá se, že hodně aktivním konvertorem modelů do EXL2 je [LoneStriker](https://huggingface.co/LoneStriker).

## Další dělení 
### Dělení dle velikosti
Ve jménech modelů se těsně na názvem rodiny objevuje něco v duchu 7b, 13b, 70b atd. Jedná se o celkový počet parametrů (vah) modelu, obvykle v miliardách (od toho pochází i písmeno b - billion). Větší číslo znamená větší sílu modelu (minimálně v rámci dané generace modelu), vykoupenou ale většími HW nároky. 

### Dělení dle délky kontextu
Velikost kontextu de facto udává velikost paměti modelu. Jedná se o maximální možný součet počtu tokenů do modelu vstupujících a z modelu vystupujících. Pokud by byl počet tokenů větší než toto číslo, bude odpověď modelu uříznutá. Měli bychom zdůraznit, že délka kontextu není totožná s pamětí u chatbotů založených na LLM - tam jde nejčastěji o počet zapamatovaných výměn uživatel - AI.  
Velikost kontextu se ne vždy objevuje ve jméně modelu. Pokud tam je, udává se v tisících a má za sebou písmeno "k". Nejtypičtější je asi velikost 4k. Existují metody, jak toto číslo navýšit, to si ale vybírá daň v podobě HW nároků.

### Dělení dle podoby promptu
Různé modely vyžadují různé formáty promptu alias tagy oddělující systémový prompt, uživatelský prompt a odpověď modelu (srovnejme např. sekci "Chat Format" [odtud](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct) se sekcí "Prompt Format" [zde](https://huggingface.co/NousResearch/Hermes-2-Pro-Llama-3-70B)). Sice formátů promtů existuje mnohem méně než modelů, ale když člověk použije formát špatný, dostane namísto rozumné odpovědi nesmysly anebo vůbec nic. Proto je vždy vhodné podívat se na kartu modelu.

### Dělení dle (ne)cenzurovanosti
Většina modelů od korporací je cenzurovaných, tj. modely odmítnou odpovídat na otázky s kriminální či NSFW tématikou. Necenzurované modely vznikly fine-tuningem těchto cenzurovaných modelů.

### Nevešlo se jinam
Existuje řada benchmarků, s pomocí kterých se mezi sebou jednotlivé modely porovnávají (např. [MMLU](https://en.wikipedia.org/wiki/MMLU)). Bohužel často dochází k tomu, že (vědomky či nevědomky) otázky z benchmarků proniknou do trénovacích dat, čímž je taková metrika poněkud znehodnocena. Též se objevily pokusy v rámci jednoho benchmarku srovnávat hrušky s jablky, popřípadě hledat specifický způsob dotazování, při kterém se model A jeví lépe než model B, přestože při nomrálním testování by srovnávání skončilo naopak. Existuje ale jeden benchmark, který není založený na odpovídání na předem známé otázky, nýbrž na hodnocení člověka - [LMSYS Chatbot Arena](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard). Uživatel položí ve webovém rozhraní otázku a dostaně odpověď ode dvou náhodně vybraných modelů. Jeden označí jako lepší (resp. řekne, že výsledky byly ekvivaletní) a až poté uvidí, jaké že modely mu vlastně odpovídaly.  

Za pozornost stojí i žebříčky, které na Redditu dělá (možná spíše dělal - dlouho se nic nového neobjevilo) uživatel [WolframRavenwolf](https://www.reddit.com/r/LocalLLaMA/comments/1cal17l/llm_comparisontest_llama_3_instruct_70b_8b/). Otázky jsou neveřejné, přičemž položeny jsou v němčině (metologie bývá na začátku příspěvku). Lze ocenit, že k jednotlivým hodnocením modelů je připojen vždy i krátký komentář.  

V názvu některých modelů člověk občas objeví písmena DPO. Jedná se o zkratku pro ["Direct Preference Optimization"](https://arxiv.org/abs/2305.18290). Víceméně se jedná o způsob, jak se zbavit pomocného modelu, který v rámci RLHF (Reinforcement Learning from Human Feedback) dotrénovává velký jazykový model s cílem produkovat člověku libé výstupy. Onen pomocný model prý totiž bývá nestabilní a poměrně komplikovaný na vytvoření. V rámci DPO dochází k RLHF v rámci trénování samotného jazykového modelu. 

Na kartách některých modelů člověk nalezne informaci, že se jedná o "frankenmerge". Víceméně jde o to, že se vrstvy několika modelů (anebo třeba i toho samého modelu) postaví na sebe, případně se trochu zdrcnou. Výsledek by údajně měl být lepší co se přirozeného toku řeči týče, avšak horší z hlediska faktografické správnosti.


## Llama cpp
Pro práci s GGUF soubory použijeme balíček [llama-cpp-python](https://pypi.org/project/llama-cpp-python/). Jedná se o pythoní wrapper kolem knihovny [llama.cpp](https://github.com/ggerganov/llama.cpp). Pokud budeme na práci s modely používat puze CPU, má instalace klasickou podobu "pip install balíček". Pro využití GPU je však potřeba
- mít NVIDIA grafickou kartu
- nainstalovat odpovídající verzi CUDY
- v příkazovém řádku postupně pustit 'set LLAMA_CUBLAS=1', 'set FORCE_CMAKE=1', 'set CMAKE_ARGS=-DCMAKE_GENERATOR_TOOLSET="cuda=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5"' (resp. obdobu závisející na adresáři a verzi použité CUDY)
- samotnou instalaci realizovat s příkazem 'pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir'

Testování provedeme na [GGUF mutaci modelu Phi-3](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf).   
Nejprve si vytvoříme objekt reprezentující velký jazykový model. V něm nejdůležitějším parametrem je *model_path*, kam vkládáme název souboru s modelem.  
Defaultně se předpokládá, že model poběží pouze na CPU. V takovém případe je *n_gpu_layers* položeno rovno nule. Tento parametr totiž říká, kolik vrstev (ve smyslu layers) modelu se má nahrát do GPU. Pokud chceme nahrát všechny, položíme parametr rovný -1.  
Pro specifikaci, kolik CPU vláken se má dát modelu k dispozici použijeme parametr *n_threads*. Hádám, že defaultní None znamená všechny. Bacha - jako max číslo pro tohle nemůžeme brát "Threads" z task managera, nýbrž spíše půlku z počtu logických procesorů - viz následující řádek z llama-cpp-python:
```python
self.n_threads = n_threads or max(multiprocessing.cpu_count() // 2, 1)
```
Člověk by si myslel, že aby model na stejný uživatelův vstup reagoval vždy stejně, museli bychom zafixovat hodnotu parametru *seed*. Defaultně je tato hodnota fixovaná skrze *llama_cpp.LLAMA_DEFAULT_SEED*. Pokud bychom chtěli chceme náhodný seed, museli bychom parametr položit rovný -1. Fakticky ale i pro seed fixovaný *llama_cpp.LLAMA_DEFAULT_SEED* dostáváme různé výsledky (a i když celou dobu kontrolujeme, zda má proměnná opravdu stejnou hodnotu)   
Parametr *n_ctx* by měl udávat maximální povolený součet délky vstupu a výstupu modelu v tokenech. Defaultní hodnota je 512. Když hodnotu příliš snížíme, můžeme dostat error v podobě "IndexError: index 10 is out of bounds for axis 0 with size 10" (to když jsem nastavil *n_ctx* = 10).    
Parametry mající v názvů *rope* souvisí v umělém prodloužení kontextu modelu. Nikdy jsem to nezkoušel použít (slabý HW).   

In [1]:
from llama_cpp import Llama

path_to_model = "Phi-3-mini-4k-instruct-q4.gguf"

llm = Llama(
  model_path=path_to_model,  
  n_gpu_layers=-1
)

llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count_kv u32           

Samotné použití modelu nastává až při jeho provolání. Nejdůležitějším parametrem je *prompt*, do kterého napíšeme uživatelům dotaz/příkaz. Obvykle je ale potřeba jej obohatit o tagy specifikující, kde že to vlasntě uživatelův vstup začíná a končí a kde by měla začínat odpověď modelu. Bez takovýchto věcí může odpověď obsahovat nesmysly anebo naopak bude zcela prázdná. Správná podoba tagů se liší model od modelu, přičemž ji najdeme v kartě modelu na Hugging facu.

In [2]:
output = llm(
  prompt="<|user|>\nWhat is hamster?<|end|>\n<|assistant|>"
)

output


llama_print_timings:        load time =     651.74 ms
llama_print_timings:      sample time =       2.23 ms /    16 runs   (    0.14 ms per token,  7165.25 tokens per second)
llama_print_timings: prompt eval time =     651.65 ms /     9 tokens (   72.41 ms per token,    13.81 tokens per second)
llama_print_timings:        eval time =     775.28 ms /    15 runs   (   51.69 ms per token,    19.35 tokens per second)
llama_print_timings:       total time =    1436.38 ms /    24 tokens


{'id': 'cmpl-d6734890-1db3-43bf-bd5a-e99824d0fab6',
 'object': 'text_completion',
 'created': 1718978920,
 'model': 'Phi-3-mini-4k-instruct-q4.gguf',
 'choices': [{'text': ' A hamster is a small rodent belonging to the subfamily Cricet',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 9, 'completion_tokens': 16, 'total_tokens': 25}}

Níže se nalézá ukázka výstupu bez tagů.

In [3]:
output = llm(
  prompt="What is hamster?"
)

output

Llama.generate: prefix-match hit

llama_print_timings:        load time =  170174.67 ms
llama_print_timings:      sample time =       0.12 ms /     1 runs   (    0.12 ms per token,  8474.58 tokens per second)
llama_print_timings: prompt eval time =  152432.03 ms /     5 tokens (30486.41 ms per token,     0.03 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  152433.38 ms /     6 tokens


{'id': 'cmpl-fbd009a5-540a-427d-8a58-dc72b837927f',
 'object': 'text_completion',
 'created': 1717940277,
 'model': 'Phi-3-mini-4k-instruct-q4.gguf',
 'choices': [{'text': '',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 6, 'completion_tokens': 0, 'total_tokens': 6}}

Důležitým parametrem je **max_tokens** říkající, kolik tokenů může maximálně zabrat odpověď. Defaultní hodnota je rovna 16 (opravdu tak málo). Pokud jí položíme rovnou None či číslu menšímu než 1, bude maximální počet vygenerovaných tokenů neomezený, resp. omezený pouze hodnotou **n_ctx** z konstruktoru modelu. V případě, že se odpověď do počtu tokenů nevejde, bude useklá a **finish_reason** se namísto klasického **stop** bude rovnat **lenght**.

In [4]:
output = llm(
  prompt="<|user|>\nWhat is hamster?<|end|>\n<|assistant|>",
  max_tokens=5,
)

output

Llama.generate: prefix-match hit

llama_print_timings:        load time =  170174.67 ms
llama_print_timings:      sample time =       0.64 ms /     5 runs   (    0.13 ms per token,  7824.73 tokens per second)
llama_print_timings: prompt eval time =  162739.85 ms /     8 tokens (20342.48 ms per token,     0.05 tokens per second)
llama_print_timings:        eval time =     299.64 ms /     4 runs   (   74.91 ms per token,    13.35 tokens per second)
llama_print_timings:       total time =  163044.52 ms /    12 tokens


{'id': 'cmpl-7f93153f-cc58-41b3-bb6a-7f7876ef40ca',
 'object': 'text_completion',
 'created': 1717940630,
 'model': 'Phi-3-mini-4k-instruct-q4.gguf',
 'choices': [{'text': ' A hamster is a',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 9, 'completion_tokens': 5, 'total_tokens': 14}}

Pokud v odpovědi chceme mít i původní prompt, použijeme parametr **echo** s hodnotou True (defaultně je tento parametr roven False).

In [5]:
output = llm(
  prompt="<|user|>\nWhat is hamster?<|end|>\n<|assistant|>",
  max_tokens=512,
  echo=True
)

output

Llama.generate: prefix-match hit

llama_print_timings:        load time =  170174.67 ms
llama_print_timings:      sample time =      67.73 ms /   377 runs   (    0.18 ms per token,  5566.38 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   64274.95 ms /   377 runs   (  170.49 ms per token,     5.87 tokens per second)
llama_print_timings:       total time =   64768.95 ms /   377 tokens


{'id': 'cmpl-3ea16bdc-6ce0-404a-b433-eada18ca98cc',
 'object': 'text_completion',
 'created': 1717940910,
 'model': 'Phi-3-mini-4k-instruct-q4.gguf',
 'choices': [{'text': "<|user|>\nWhat is hamster?<|end|>\n<|assistant|> A hamster is a small, rodent mammal belonging to the family Cricetidae. There are approximately 19 different species of hamsters found naturally in various regions around the world, with one species, the golden hamster (Campbell's dwarf hamster), being commonly kept as pets globally due to its docile nature and manageable size.\n\nHamsters have round bodies, short limbs, and large cheek pouches, which are used for storing food. They come in a variety of colors (including golden, white, black, brown, or even multi-colored) and sizes but typically range from 5 to 12 inches long with an additional tail length ranging between 0.3 to 0.8 inches.\n\nIn the wild, hamsters live in burrows they dig underground for shelter from predators and extreme temperatures. Their diets ma

Přidat lze i parametr **stop**. Do něj vložíme list s tagy, které - objeví-li se ve výstupu modelu - povedou k ukončení generování. Obvykle se jedná o "</s>" či "<|end|>" - konkrétní tag člověk opět najde v kartě modelu. Defaultní hodnotou je prázdný list.  
Míru chaotičnosti výstupu můžeme ovlivnit s parametry **temperature** (default 0.8, čím větší hodnota, tím větší chaotičnost a **top_p** (default 0.95, čím menší hodnota, tím větší chaotičnost).  
I zde se objevuje parametr **seed**, který tentokrát opravdu vede k fixaci výstupu.